# Setup
Specify the paths you chose for your models. Note: this will be relative to where
this notebook is.

In [5]:
import numpy as np
import json

from numpy import ndarray
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

def subplot_3d():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    return fig, ax

def draw_arrows(ax: Axes, locs: ndarray, dirs: ndarray, scale: float = 0.1):
    handles = []
    for bp, s in zip(locs, dirs):
        if is_3d:
            handles.append(ax.quiver(*bp, *(s * scale), color='black'))
        else:
            print(bp)
            print(s)
            handles.append(ax.arrow(*bp, *(s * scale), head_width=scale * 0.1, head_length=scale*0.2))

def subplot():
    if is_3d:
        fig, ax = subplot_3d()
    else:
        fig, ax = plt.subplots()

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    return fig, ax
    
def load_boundary(path) -> ndarray:
    with open(path) as f: # TODO
        data = json.load(f)
    return np.array(data['boundary']["boundary_points"]), np.array(data['boundary']["boundary_surface"])
    


In [6]:
# Load boundary
path = "/home/bentoaz/SEMBAS/.results/old_bounds/vector_pass/boundary1.json" # TODO set to the full path of your boundary folder
bpoints, bdir = load_boundary(path)

ndim = bdir.shape[1]
is_3d = ndim == 3

assert ndim <= 3, "Cannot visualize more than 3 dimensions"

print(len(bpoints.T))
print(bpoints.shape, bdir.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/home/bentoaz/SEMBAS/.results/old_bounds/vector_pass/boundary1.json'

In [ ]:
# # Show boundary
fig, ax = subplot()

ax.scatter(*bpoints.T, color="red")
plt.show()

# show surface direction
fig, ax = subplot()

ax.scatter(*bpoints.T, color="red")
draw_arrows(ax, bpoints, bdir)
plt.show()

In [ ]:
import numpy as np
import json
import math
from numpy import ndarray
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

# Global variable to control 2D/3D plotting
is_3d = True

def subplot_3d():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    return fig, ax

def draw_arrows(ax: Axes, locs: ndarray, dirs: ndarray, scale: float = 0.1):
    handles = []
    for bp, s in zip(locs, dirs):
        if is_3d:
            handles.append(ax.quiver(*bp, *(s * scale), color='black'))
        else:
            print(bp)
            print(s)
            handles.append(ax.arrow(*bp, *(s * scale), head_width=scale * 0.1, head_length=scale*0.2))
    return handles

def subplot():
    if is_3d:
        fig, ax = subplot_3d()
    else:
        fig, ax = plt.subplots()
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
    return fig, ax

def load_boundary(path) -> tuple:
    with open(path) as f:
        data = json.load(f)
    return np.array(data['boundary']["boundary_points"]), np.array(data['boundary']["boundary_surface"])

def plot_all_jsons_in_directory(directory_path, start_index=0, num_plots=None):
    """
    Plot all JSON files in a directory in one shot
    """
    import os
    import glob
    
    # Find all JSON files in the directory
    json_files = glob.glob(os.path.join(directory_path, "*.json"))
    
    if not json_files:
        print(f"No JSON files found in directory: {directory_path}")
        return
    
    print(f"Found {len(json_files)} JSON files:")
    for file in json_files:
        print(f"  - {os.path.basename(file)}")
    
    # Combine all data from all JSON files
    all_data = []
    file_sources = []  # Keep track of which file each data point came from
    
    for json_file in sorted(json_files):  # Sort for consistent ordering
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            # Handle both single objects and arrays
            if isinstance(data, list):
                all_data.extend(data)
                file_sources.extend([os.path.basename(json_file)] * len(data))
            else:
                all_data.append(data)
                file_sources.append(os.path.basename(json_file))
                
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Warning: Could not load {json_file}: {e}")
            continue
    
    if not all_data:
        print("No valid data found in JSON files")
        return
    
    print(f"\nLoaded {len(all_data)} data points total")
    
    # Determine number of plots
    if num_plots is None:
        num_plots = len(all_data) - start_index
    
    # Group plots into pages of 4
    plots_per_page = 4
    num_pages = math.ceil(num_plots / plots_per_page)
    
    for page in range(num_pages):
        # Calculate which plots go on this page
        page_start = start_index + (page * plots_per_page)
        page_end = min(page_start + plots_per_page, start_index + num_plots)
        
        # Create subplot grid: 2 rows, 2 columns
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        if is_3d:
            # For 3D plots, we need to recreate the subplots with 3D projection
            fig = plt.figure(figsize=(16, 12))
            axes = []
            for i in range(4):
                row = i // 2
                col = i % 2
                ax = fig.add_subplot(2, 2, i+1, projection='3d')
                axes.append(ax)
            axes = np.array(axes).reshape(2, 2)
        
        fig.suptitle(f"All JSON Results Grid - Page {page + 1}/{num_pages}", fontsize=16)
        
        # Plot positions for 2x2 grid
        positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
        
        # Use original function logic in a for loop
        for idx, plot_num in enumerate(range(page_start, page_end)):
            if plot_num >= len(all_data):
                break
            
            # Original function logic starts here
            data_point = all_data[plot_num]
            source_file = file_sources[plot_num]
            
            # Handle different data structures
            if "requests" in data_point and "results" in data_point:
                # Original structure with requests/results
                coords = data_point["requests"]
                results = data_point["results"]
                description = data_point.get("description", f"Data from {source_file}")
                
                # Extract coordinates
                x = [pt[0] for pt in coords]
                y = [pt[1] for pt in coords]
                z = [pt[2] for pt in coords] if is_3d else None
                
                # Create color mapping (0 = red/failed, 1 = blue/successful)
                colors = ['red' if not res else 'blue' for res in results]
                
            elif "boundary" in data_point:
                # Boundary structure with boundary_points
                boundary_points = data_point["boundary"]["boundary_points"]
                description = data_point.get("description", f"Boundary from {source_file}")
                volume = data_point.get("volume", "Unknown")
                
                # Extract coordinates from boundary points
                x = [pt[0] for pt in boundary_points]
                y = [pt[1] for pt in boundary_points]
                z = [pt[2] for pt in boundary_points] if is_3d else None
                
                # Use a single color for boundary points (or gradient based on index)
                colors = ['blue'] * len(boundary_points)  # All blue for boundary points
                # Or create a gradient: colors = plt.cm.viridis(np.linspace(0, 1, len(boundary_points)))
                
                # Update description to include volume
                description = f"{description}\nVolume: {volume}"
                
            else:
                # Fallback for unknown structure
                print(f"Warning: Unknown data structure in {source_file}")
                continue
            
            # Get the appropriate subplot
            row, col = positions[idx]
            ax = axes[row, col] if isinstance(axes, np.ndarray) else axes[idx]
            
            # Create the same scatter plot as original function
            if is_3d:
                scatter = ax.scatter(x, y, z, c=colors, s=60, alpha=1.0, 
                                  edgecolors='black', linewidth=1)
                ax.set_xlabel('Progress Weight')
                ax.set_ylabel('Bounds Weight')
                ax.set_zlabel('Proximity Weight')
            else:
                scatter = ax.scatter(x, y, c=colors, s=60, alpha=1.0, 
                                  edgecolors='black', linewidth=1)
                ax.set_xlabel('Progress Weight')
                ax.set_ylabel('Bounds Weight')
                ax.set_xlim(0, 1)
                ax.set_ylim(0, 1)
            
            # Include source file in title
            title = f"{description}\n(from {source_file})"
            ax.set_title(title, fontsize=9, pad=10)
            
            # Add arrows if needed (using your draw_arrows function)
            # Uncomment and modify as needed:
            if 'arrows' in data_point:
                arrow_locs = np.array(data_point['arrow_locs'])
                arrow_dirs = np.array(data_point['arrow_dirs'])
                draw_arrows(ax, arrow_locs, arrow_dirs)
        
        # Hide empty subplots
        total_plots_this_page = page_end - page_start
        for idx in range(total_plots_this_page, 4):
            row, col = positions[idx]
            if isinstance(axes, np.ndarray):
                axes[row, col].set_visible(False)
            else:
                axes[idx].set_visible(False)
        
        plt.tight_layout()
        plt.show()

In [ ]:
path = "/home/bentoaz/SEMBAS/.results/old_bounds/scalar_pass"
plot_all_jsons_in_directory(path)

In [8]:
import numpy as np
import json
import math
from numpy import ndarray
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import glob

def draw_arrows(fig, locs: ndarray, dirs: ndarray, scale: float = 0.1, row=None, col=None):
    """Add 3D arrows to plotly figure"""
    for bp, s in zip(locs, dirs):
        end_point = bp + (s * scale)
        
        # Arrow line
        fig.add_trace(
            go.Scatter3d(
                x=[bp[0], end_point[0]],
                y=[bp[1], end_point[1]], 
                z=[bp[2], end_point[2]],
                mode='lines',
                line=dict(color='black', width=4),
                showlegend=False,
                hoverinfo='skip'
            ),
            row=row, col=col
        )
        
        # Arrow head
        fig.add_trace(
            go.Scatter3d(
                x=[end_point[0]],
                y=[end_point[1]],
                z=[end_point[2]],
                mode='markers',
                marker=dict(color='black', size=8, symbol='diamond'),
                showlegend=False,
                hoverinfo='skip'
            ),
            row=row, col=col
        )

def load_boundary(path) -> tuple:
    with open(path) as f:
        data = json.load(f)
    return np.array(data['boundary']["boundary_points"]), np.array(data['boundary']["boundary_surface"])

def plot_single_json(data_point, source_file=""):
    """Create interactive 3D plot for a single data point"""
    
    # Handle different data structures
    if "requests" in data_point and "results" in data_point:
        coords = data_point["requests"]
        results = data_point["results"]
        description = data_point.get("description", f"Data from {source_file}")
        
        x = [pt[0] for pt in coords]
        y = [pt[1] for pt in coords]
        z = [pt[2] for pt in coords]
        
        colors = ['red' if not res else 'blue' for res in results]
        hover_text = [f"Point {i}<br>({pt[0]:.3f}, {pt[1]:.3f}, {pt[2]:.3f})<br>{'Success' if res else 'Failed'}"
                     for i, (pt, res) in enumerate(zip(coords, results))]
        
    elif "boundary" in data_point:
        boundary_points = data_point["boundary"]["boundary_points"]
        description = data_point.get("description", f"Boundary from {source_file}")
        volume = data_point.get("volume", "Unknown")
        
        x = [pt[0] for pt in boundary_points]
        y = [pt[1] for pt in boundary_points]
        z = [pt[2] for pt in boundary_points]
        
        colors = ['blue'] * len(boundary_points)
        hover_text = [f"Boundary {i}<br>({pt[0]:.3f}, {pt[1]:.3f}, {pt[2]:.3f})"
                     for i, pt in enumerate(boundary_points)]
        
        description = f"{description}<br>Volume: {volume}"
        
    else:
        raise ValueError(f"Unknown data structure in {source_file}")
    
    # Create 3D scatter plot
    fig = go.Figure(data=go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            color=colors,
            size=8,
            line=dict(color='black', width=1)
        ),
        text=hover_text,
        hoverinfo='text'
    ))
    
    # Add arrows if present
    if 'arrows' in data_point:
        arrow_locs = np.array(data_point['arrow_locs'])
        arrow_dirs = np.array(data_point['arrow_dirs'])
        draw_arrows(fig, arrow_locs, arrow_dirs)
    
    # Set layout
    title = description
    if source_file:
        title += f"<br><sub>{source_file}</sub>"
        
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='Progress Weight',
            yaxis_title='Bounds Weight',
            zaxis_title='Proximity Weight'
        ),
        width=800,
        height=600
    )
    
    return fig

def plot_all_jsons_in_directory(directory_path, start_index=0, num_plots=None):
    """Plot all JSON files in directory as individual interactive 3D plots"""
    
    # Find JSON files
    json_files = glob.glob(os.path.join(directory_path, "*.json"))
    
    if not json_files:
        print(f"No JSON files found in directory: {directory_path}")
        return []
    
    print(f"Found {len(json_files)} JSON files")
    
    # Load all data
    all_data = []
    file_sources = []
    
    for json_file in sorted(json_files):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            if isinstance(data, list):
                all_data.extend(data)
                file_sources.extend([os.path.basename(json_file)] * len(data))
            else:
                all_data.append(data)
                file_sources.append(os.path.basename(json_file))
                
        except Exception as e:
            print(f"Warning: Could not load {json_file}: {e}")
            continue
    
    if not all_data:
        print("No valid data found")
        return []
    
    print(f"Loaded {len(all_data)} data points total")
    
    # Create plots
    if num_plots is None:
        num_plots = len(all_data) - start_index
    
    figures = []
    for plot_num in range(start_index, min(start_index + num_plots, len(all_data))):
        data_point = all_data[plot_num]
        source_file = file_sources[plot_num]
        
        try:
            fig = plot_single_json(data_point, source_file)
            figures.append(fig)
        except Exception as e:
            print(f"Warning: Could not plot data point {plot_num}: {e}")
            continue
    
    return figures

# Simple usage
def show_plots(directory_path, start_index=0, num_plots=None):
    """Show all plots"""
    figures = plot_all_jsons_in_directory(directory_path, start_index, num_plots)
    
    for i, fig in enumerate(figures):
        print(f"Showing plot {i + 1}/{len(figures)}")
        fig.show()

# Example usage:
# show_plots("path/to/json/directory")
# 
# Or for a single file:
# with open("data.json") as f:
#     data = json.load(f)
# fig = plot_single_json(data, "data.json")
# fig.show()

In [9]:
# Show all JSON files in a directory
path = "/home/bentoaz/SEMBAS/.results/old_bounds/vector_bounds"
show_plots(path)

# Show specific range
# show_plots("path/to/json/directory", start_index=2, num_plots=5)

Found 3 JSON files
Loaded 3 data points total
Showing plot 1/3


Showing plot 2/3


Showing plot 3/3
